In [ ]:
result = {
    'product_pattern': {'ocr': '아직없음', 'db': '아직없음', 'score': 1.0, 'result': '승인'},
  'product_name': {
    'ocr': '친환경 완숙 토마토',
    'db': '친환경 대추방울토마토',
    'score': 0.6666666666666666,
    'incorrect_letter': {
      4: '완',
      5: '숙'
    },
    'result': '오류',
    'reason': '인식 결과 95% 미만'
  },
  'weight': {
    'ocr': '1k지',
    'db': '600g',
    'result': '매칭실패',
    'score': 0,
    'reason': '매칭실패'
  },
  'barcode': {
    'ocr': '2500000279942',
    'db': '2500000279942',
    'score': 1.0,
    'result': '승인',
    'reason': ''
  },
  'cert_mark': {
    'ocr': '무농약',
    'db': [
      '3'
    ],
    'result': '승인',
    'score': 1.0,
    'reason': [
      ''
    ]
  },
  'cert_nums': [
    {
      'ocr_num': '17303151',
      'db_num': '17303151',
      'num_score': 1.0,
      'ocr_name': '진환경아리작육반',
      'db_name': '친환경아라작목반',
      'name_score': 0.625,
      'incorrect': {
        0: '진',
        4: '리',
        6: '육'
      },
      'result': '승인',
      'reason': ''
    }
  ]
}

In [ ]:
    def check_cert_information(self, cert_lst, num_score, name_score):
        total_result = []
        for idx, cert_info in enumerate(cert_lst):
            cert_num, cert_name = self.check_num_str(cert_info)
            info_result = {}
            gubun_dic = {}
            db_names = list(self.cert_num_db['눈으로보고고치기'])
            db_nums = list(self.cert_num_db['빼기기호없애기'])
            
            # 승인
            # OCR을 통해 완전한 인증코드를 획득하고 OCR생산자명과 인증등록정보 생산자명이 90%미만 일치
            # OCR을 통해 완전한 인증코드를 획득했다고 생각했으나 인증등록정보에 없는 코드인 경우
            if cert_num in list(db_nums):
                index = db_nums.index(cert_num)
                db_name = db_names[index]
                info_result['cert_num'] = {'ocr': cert_num, 'db': db_nums[cert_num], 'score': 1.0, 'incorrect_letter': None}
                info_result['cert_name'] = self.similar([cert_name], db_name)
                if int(info_result['cert_name']['score'] * 100) >= name_score:
                    result = '승인'
                else:
                    result = '오류'
                info_result['result'] = result
            else:
                for db_name, db_num in zip(cert_name, cert_num):
                    score, top_score = self.get_cert_num_score(cert_num, db_num)
                    gubun_dic[db_num] = [score, db_name]

                if len([k for k, v in gubun_dic.items() if v[0] == top_score]) != 1:
                    info_result['result'] = '인식실패'
                else:
                    if top_score > num_score:
                        top_num = [k for k, v in gubun_dic.items() if v[0] == top_score][0]
                        top_text = [v[1] for k, v in gubun_dic.items() if v[0] == top_score][0]
                        info_result['ocr_num'] = cert_num
                        info_result['db_num'] = top_num
                        info_result['num_score'] = top_score
                        name_result = self.similar([cert_name], top_text)
                        info_result['ocr_name'] = cert_name
                        info_result['db_name'] = top_text
                        info_result['name_score'] = name_result['score']
                        info_result['incorrect'] = name_result['incorrect_letter']
                        info_result['result'] = '승인'
                        info_result['reason'] = ''
                    else:
                        info_result['reason'] = '인식결과가 기준보다 낮아요'
                        info_result['result'] = '매칭실패'
            total_result.append(info_result)
        return total_result